In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install libtiff
!pip install geopandas  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 4.4 MB/s 
  Created wheel for libtiff: filename=libtiff-0.4.2-cp37-cp37m-linux_x86_64.whl size=285109 sha256=b85b80a3d0b7695df473346dc5398e906c417b746e538e9bb61d410e32113425
  Stored in directory: /root/.cache/pip/wheels/ea/3d/1d/9ad926b88ecfd8719065c57ad36808810a77123c4647e8d341
Successfully built libtiff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.3 MB 27.2 MB/s 
     |████████████████████████████████| 16.7 MB 27.3 MB/s 


In [ ]:
import json                                                                                          
import sys 
import rasterio
from rasterio.mask import mask
from json import loads
import sys 
import os
from libtiff import TIFF                                                                             
import sys 
import os
from PIL import Image
from os import listdir
from os.path import isfile, join
import pickle
import h5py
import scipy.misc
import glob
import imageio
from os import listdir
from os.path import isfile, join
import cv2
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import geopandas as gpd

In [ ]:
def cut_villages(dist):
  print('Cutting villages for: ',dist)

  path = '/content/drive/My Drive/Landsat7_'+dist.replace(' ','')
  print('GEE image of district at: ',path)

  os.chdir(path)

  fnames = []
  for f in listdir():
    if(f != '.ipynb_checkpoints' and f!='Villages'):
      fnames.append(f)

  print('Number of images of district: ',len(fnames))
  #print(fnames)

  #_201: because years are 2014-2019
  n = fnames[0].find('_201')

  #read the geojson file of the district
  fname1 = '/content/'+dist+'.geojson'
  print('District filename: ',fname1)
  df = gpd.read_file(fname1)
  print('Number of villages: ',len(df))
  
  fname = '/content/drive/MyDrive/Landsat7_'+ dist.replace(' ','')+'/'
  print(fname)

  for f in tqdm(fnames):
    for i in range(len(df)):
      geoms = [df['geometry'][i]]
      with rasterio.open(fname+f) as src:
        out_image,out_transform = mask(src,geoms,crop=True)
      
      out_meta = src.meta.copy()
      out_meta.update({"driver": "GTiff",
          "height": out_image.shape[1],
          "width": out_image.shape[2],
          "transform": out_transform})
      
      try:
        #temp = df['VILL_NAME'][i] +'_'+str(df['pc11_village_id'][i])
        #for UP: instead of pc11_village_id, it is pc11_villa
        temp = df['VILL_NAME'][i] +'_'+str(df['pc11_villa'][i])
        temp = temp.replace(' ','')
      except:
        temp = 'nf'

      if(os.path.isdir(fname + 'Villages/'+temp) == False):
        os.makedirs(fname + 'Villages/'+temp)

      opfilename = fname + 'Villages/'+temp+'/'+temp+'_'+f[n+1:-4]+'.tif'
      try:
        with rasterio.open(opfilename, "w", **out_meta) as dest:
          dest.write(out_image)
      except:
          print('error writing image!')    


In [ ]:
count = 0
for dist in os.listdir('/content/'):
  if('.geojson' in dist):
    dist = dist[:-8]
    print(dist)
    #this condition for cases like: Saraikela-kharsawan
    '''if('-' in dist):
      dist = dist[0].upper() + dist[1:]
    else:
      dist = dist.title()
    ''' 
    try:
      cut_villages(dist)
      count = count + 1
    except Exception as exp:
      print('Exception for district: ',dist)
      print(exp)
print(count)

Sonbhadra
Cutting villages for:  Sonbhadra
GEE image of district at:  /content/drive/My Drive/Landsat7_Sonbhadra
Number of images of district:  21
District filename:  /content/Sonbhadra.geojson
Number of villages:  1288
/content/drive/MyDrive/Landsat7_Sonbhadra/


  0%|          | 0/21 [00:00<?, ?it/s]

1
